## Data Preprocessing for Multi-Class Classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive/P1_Final/"
!pip install import-ipynb
import import_ipynb

# importing helper functions for data preprocessing and model visualization
import plotlib_helper as graphplot
import datapreprocess_helper as dataprep

/content/drive/My Drive/P1_Final
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=33c44fa626fbe0ad2538ca024252d81f9030964555f2a27321e5b32cbe0a47ce
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
importing Jupyter notebook from plotlib_helper.ipynb
importing Jupyter notebook from datapreprocess_helper.ipynb


## Load Data

In [ ]:
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "/content/drive/My Drive/P1_Final/"
    
filepath_train = os.path.join(path,"data/UNSW_NB15_training-set_csc215.csv") 
filepath_test = os.path.join(path,"data/UNSW_NB15_test-set_csc215.csv") 

df_train= pd.read_csv(filepath_train, na_values=['NA','?','-'])
df_test= pd.read_csv(filepath_test, na_values=['NA','?','-'])

print(df_train.shape)
print(df_test.shape)



(175341, 45)
(82332, 45)


## Data Preprocessing


*   Dropped rows with null values
*   Dropped id column
*   One hot encoded categorical columns ['proto', 'state', 'service', 'attack_cat']
*   Normalized numeric fields
*   Converted input feature and target into numpy array for TensorFlow







In [ ]:
# dropping any rows with null values in training and testing data
df_train.dropna(axis=0, inplace=True)
df_test.dropna(axis=0, inplace=True)

print(df_train.shape)
print(df_test.shape)

# drop id column
df_train.drop(columns=['id'], axis= 1, inplace=True)
df_test.drop(columns=['id'], axis= 1, inplace=True)

df_train[0:5]

(81173, 45)
(35179, 45)


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2.740179e+03,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
11,2.093085,tcp,smtp,FIN,62,28,56329,2212,42.520967,62,252,2.118251e+05,8152.559082,28,8,34.312868,75.092445,3253.278833,106.113453,255,1824722662,860716719,255,0.131140,0.052852,0.078288,909,79,0,0,2,1,1,1,1,2,0,0,0,1,1,0,Normal,0
15,0.000002,udp,snmp,INT,2,0,138,0,500000.001300,254,0,2.760000e+08,0.000000,0,0,0.002000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,69,0,0,0,1,2,1,1,1,4,0,0,0,2,1,0,Normal,0
17,0.393556,tcp,http,FIN,10,8,860,1096,43.195886,62,252,1.573347e+04,19494.048830,2,2,43.728444,47.669145,2124.837873,71.805867,255,3882971404,3084071099,255,0.120313,0.059201,0.061112,86,137,1,103,3,1,1,1,1,2,0,0,1,1,3,0,Normal,0
21,0.338017,tcp,http,FIN,10,6,998,268,44.376468,254,252,2.127704e+04,5301.508789,2,1,35.218111,56.579801,1928.550710,82.022258,255,2665974075,3521361798,255,0.154433,0.055109,0.099324,100,45,1,0,6,1,1,1,1,1,0,0,1,2,3,0,Normal,0


In [ ]:
df_test [0:5]

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
35,0.983874,tcp,http,FIN,10,8,816,1172,17.278635,62,252,5976.375000,8342.531250,2,2,109.319333,124.932859,5929.211713,192.590406,255,794167371,1624757001,255,0.206572,0.108393,0.098179,82,147,1,184,2,1,1,1,1,2,0,0,1,1,3,0,Normal,0
40,1.535254,tcp,http,FIN,10,10,826,1266,12.375802,62,252,3876.882812,5940.385254,2,2,170.481889,159.070562,11933.065600,244.118016,255,708428911,3141441573,255,0.180318,0.103612,0.076706,83,127,1,187,2,1,1,1,1,2,0,0,1,1,3,0,Normal,0
45,1.059359,tcp,http,FIN,10,8,830,1134,16.047441,62,252,5641.147461,7498.874512,2,2,117.706556,132.899281,6102.668305,255.756219,255,1853677305,1202418679,255,0.274887,0.128459,0.146428,83,142,1,165,1,1,1,1,1,1,0,0,1,3,1,0,Normal,0
49,0.990548,tcp,http,FIN,10,10,804,1414,19.181301,62,252,5847.268066,10281.177730,2,2,110.060889,100.616555,7172.956819,143.497953,255,1528762476,1066585482,255,0.181441,0.084249,0.097192,80,141,1,261,2,1,18,1,1,2,0,0,1,1,2,0,Normal,0
72,1.303518,tcp,http,FIN,12,8,898,1120,14.575939,62,252,5057.083984,6014.492676,2,2,118.501635,175.467000,6491.561713,252.408328,255,1182907296,802589817,255,0.140062,0.074510,0.065552,75,140,1,157,1,1,1,1,1,1,0,0,1,2,1,0,Normal,0


In [ ]:
#one hot encoding for categorical columns
categorical_cols =['proto', 'state', 'service', 'label']
non_normalize_cols = ['is_sm_ips_ports', 'is_ftp_login','attack_cat']

for col in df_train.columns:
  if col not in categorical_cols and col not in non_normalize_cols:
    dataprep.encode_numeric_zscore(df_train, col)

for col in df_test.columns:
  if col not in categorical_cols and col not in non_normalize_cols:
    dataprep.encode_numeric_zscore(df_test, col)

for col in categorical_cols:
  dataprep.encode_text_dummy(df_train, col)
  dataprep.encode_text_dummy(df_test, col)
train_labels = df_train['attack_cat'].unique();
print(train_labels)

test_labels = df_test['attack_cat'].unique();
print(test_labels)

intersection_set = set.intersection(set(train_labels), set(test_labels))
commom_cat = list(intersection_set)
print(commom_cat)
df_train.sort_values('attack_cat', inplace=True)
df_test.sort_values('attack_cat',inplace=True)


indexNames_train = df_train[~(df_train['attack_cat'].isin(commom_cat))].index
df_train.drop(indexNames_train , inplace=True)

indexNames_test = df_test[~(df_test['attack_cat'].isin(commom_cat))].index
df_test.drop(indexNames_test , inplace=True)

print(df_train.shape, df_test.shape)
df_train['attack_cat'].value_counts()


['Normal' 'Backdoor' 'Fuzzers' 'Reconnaissance' 'Exploits' 'Analysis'
 'DoS' 'Worms' 'Generic']
['Normal' 'Exploits' 'Reconnaissance' 'Fuzzers' 'DoS' 'Worms' 'Backdoor'
 'Generic']
['DoS', 'Worms', 'Reconnaissance', 'Backdoor', 'Fuzzers', 'Normal', 'Generic', 'Exploits']
(80609, 61) (35179, 61)


Generic           39496
Normal            19488
Exploits          16187
DoS                1791
Fuzzers            1731
Reconnaissance     1703
Worms               114
Backdoor             99
Name: attack_cat, dtype: int64

In [ ]:
# retaining only the common columns
common_col = (df_train.columns) & (df_test.columns) 
df_train = df_train[common_col]
df_test = df_test[common_col]
print("Data frame shape after removing non-common columns",df_train.shape, df_test.shape)
print(df_train['attack_cat'].unique())
print(df_test['attack_cat'].unique())



Data frame shape after removing non-common columns (80609, 60) (35179, 60)
['Backdoor' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal' 'Reconnaissance'
 'Worms']
['Backdoor' 'DoS' 'Exploits' 'Fuzzers' 'Generic' 'Normal' 'Reconnaissance'
 'Worms']


In [ ]:
train_write = os.path.join(path,"data/P1_MultiClass_preprocess_TRAINSET.csv")
test_write = os.path.join(path,"data/P1_MultiClass_preprocess_TESTSET.csv")

df_train.to_csv(train_write,index=False)
df_test.to_csv(test_write,index=False)

print("Successfully saved preprocessed dataset")

Successfully saved preprocessed dataset
